In [4]:
import warnings
import sys

warnings.filterwarnings('ignore')
current_dir = %pwd

parent_dir = os.path.abspath(os.path.join(current_dir, '../..'))
sys.path.append(parent_dir)

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from datasets import Dataset

label2id = {
    "Politics": 0,
    "Voices": 1,
    "Sports": 2,
    "Entertainment": 3,
    "Life": 4
}

id2label = {int(v):str(k) for k, v in label2id.items()}

checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=5, label2id=label2id, id2label=id2label)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from src.main.models.roBERTa import RoBERTa
from src.main.pipeline.functions import clean_text, stop_words_removal
from src.main.utilities.utils import get_dataset, split_train_val_test

roberta = RoBERTa()

roberta.pipeline = [clean_text, stop_words_removal]

inputs, targets = get_dataset(one_hot=False)
inputs = roberta.run_pipeline(inputs, save=True)
targets = targets.map(label2id)

x_train, x_val, x_test, y_train, y_val, y_test = split_train_val_test(inputs, targets, validation_size=0.2, test_size=0.1)

train_data = {'text': x_train, 'label': y_train}
val_data = {'text': x_val, 'label': y_val}
test_data = {'text': x_test, 'label': y_test}

In [7]:
train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)
test_dataset = Dataset.from_dict(test_data)

def tokenize_function(batch):
    return tokenizer(batch['text'], truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/75766 [00:00<?, ? examples/s]

Map:   0%|          | 0/21648 [00:00<?, ? examples/s]

Map:   0%|          | 0/10824 [00:00<?, ? examples/s]

In [8]:
from transformers import TrainingArguments, Trainer
import numpy as np
from datasets import load_metric

metrics = {
    "accuracy": load_metric("accuracy"),
    "precision": load_metric("precision"),
    "recall": load_metric("recall"),
    "f1": load_metric("f1"),
}

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = metrics["accuracy"].compute(predictions=predictions, references=labels)
    precision = metrics["precision"].compute(predictions=predictions, references=labels, average="macro")
    recall = metrics["recall"].compute(predictions=predictions, references=labels, average="macro")
    f1 = metrics["f1"].compute(predictions=predictions, references=labels, average="macro")
    return {
        "accuracy": accuracy["accuracy"],
        "precision": precision["precision"],
        "recall": recall["recall"],
        "f1": f1["f1"],
    }

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=16,   # batch size per device during training
    per_device_eval_batch_size=16,    # batch size for evaluation
    learning_rate=2e-5,              # learning rate
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=500,               # log every 500 steps
    evaluation_strategy="epoch",     # evaluate at the end of each epoch
    save_strategy="epoch",           # save the model at the end of each epoch
    load_best_model_at_end=True,     # load the best model when finished training (if using evaluation)
    metric_for_best_model="accuracy" # metric to use to compare models
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.351900,0.345210,0.884285,0.834902,0.844509,0.839072
2,0.261100,0.356674,0.893986,0.856437,0.848981,0.852499
3,0.176400,0.435501,0.893016,0.847297,0.852703,0.849539
4,0.113800,0.526741,0.891861,0.846266,0.852232,0.848944


KeyboardInterrupt: 